In [24]:
import pandas as pd
import numpy as np
import requests
from serpapi import GoogleSearch
from bs4 import BeautifulSoup
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import pickle

In [2]:
# settings
pd.set_option('display.max_colwidth', None)

In [ ]:
# # model training

# training_query = input() + ' ' + 'AND (convicted OR charged OR money laundering OR tax evasion OR embezzle OR investigation OR trialed OR sentenced OR corruption OR fraud OR fine OR penalty OR terrorist OR tax)'

# training_query = 'company'

# training_params = {
#   "engine": "google",
#      "tbm": "nws",
#     "q": training_query, 
#     'num': '70',
#   "api_key": api_key
# }

# training_search = GoogleSearch(training_params)
# traning_results = training_search.get_dict()

In [ ]:
# training_news_results = traning_results["news_results"]

In [ ]:
# training_results_fps = pd.DataFrame(training_news_results)

In [ ]:
# training_results_fps.drop(['position'], axis=1, inplace=True)
# training_results_fps['snippet'] = training_results_fps['snippet'].str.replace('\n', '')
# training_results_fps['snippet'] = training_results_fps['snippet'].str.lower()
# training_results_fps['title'] = training_results_fps['title'].str.lower()

In [ ]:
# training_frames.to_csv('training_data_new.csv', index=False)

In [3]:
training = pd.read_csv('training_data_clean_class.csv')

In [4]:
training['snippet'] = training['snippet'].str.replace('\n', '')
training['snippet'] = training['snippet'].str.lower()
training['title'] = training['title'].str.lower()

In [5]:
training.dtypes

title                        object
link                         object
displayed_link               object
snippet                      object
snippet_highlighted_words    object
label                        object
dtype: object

In [6]:
training['label'].value_counts()

relevant        186
not relevant    116
Name: label, dtype: int64

In [7]:
training.head(20)

,title,link,displayed_link,snippet,snippet_highlighted_words,label
0,hsbc holdings plc. and hsbc bank usa n.a. admit to anti ...,https://www.justice.gov/opa/pr/hsbc-holdings-plc-and-hsbc-bank-usa-na-admit-anti-money-laundering-and-sanctions-violations,https://www.justice.gov › Office of Public Affairs › News,"according to court documents, hsbc bank usa violated the bsa by failing to maintain an effective anti-money laundering program and to conduct ...",['violated the BSA'],relevant
1,hsbc's money laundering scandal - investopedia,https://www.investopedia.com/stock-analysis/2013/investing-news-for-jan-29-hsbcs-money-laundering-scandal-hbc-scbff-ing-cs-rbs0129.aspx,https://www.investopedia.com › Investing › Stocks,hsbc holdings' (hsbc) agreement to pay a $1.9 billion fine to regulators for serving as a middleman for mexican drug cartels and enter into a deferred ...,['pay a $1.9 billion fine'],relevant
2,fed ends decade-long enforcement action against hsbc,https://www.reuters.com/business/finance/us-fed-terminates-enforcement-action-against-hsbc-2022-09-02/,https://www.reuters.com › business › finance › us-fed-ter...,"london-headquartered hsbc was accused in 2012 of degenerating into a ""preferred financial institution"" for mexican and colombian drug cartels ...",['accused in 2012 of degenerating'],relevant
3,hsbc fined £64m for anti-money laundering failings - bbc,https://www.bbc.com/news/business-59689581,https://www.bbc.com › news › business-59689581,"banking giant hsbc has been fined £63.9m by the uk's financial regulator for ""unacceptable failings"" of its anti-money laundering systems. the ...",['has been fined £63.9m by'],relevant
4,hsbc moved vast sums of dirty money after paying record ...,https://www.icij.org/investigations/fincen-files/hsbc-moved-vast-sums-of-dirty-money-after-paying-record-laundering-fine/,https://www.icij.org › investigations › fincen-files › hsbc-...,hsbc moved vast sums of dirty money after paying record laundering fine ... fincen files probe reveals europe's biggest bank aided massive ponzi ...,"['HSBC', 'money', 'laundering fine']",relevant
5,fca fines hsbc bank plc £63.9 million for deficient ...,https://www.fca.org.uk/news/press-releases/fca-fines-hsbc-bank-plc-deficient-transaction-monitoring-controls,https://www.fca.org.uk › News,we have fined hsbc bank plc (hsbc) £63946800 for failings in its anti-money laundering processes.,['£63946800'],relevant
6,hsbc faces questions over disclosure of alleged money ...,https://www.theguardian.com/business/2021/jul/28/hsbc-faces-questions-over-disclosure-of-alleged-money-laundering-to-monitors,https://www.theguardian.com › business › jul › hsbc-fa...,the uk's financial conduct authority launches an investigation into hsbc over potential breaches of money laundering rules over concerns were ...,"['investigation', 'HSBC', 'money laundering']",relevant
7,hsbc to pay $192m penalty in us tax evasion case,https://www.ft.com/content/e7d51ec4-1b99-11ea-97df-cc63de1d73f4,https://www.ft.com › ... › Financials › HSBC Holdings PLC,hsbc's private bank will pay $192m in penalties and admit to helping us customers hide more than $1bn in assets from tax authorities as part ...,['will pay $192m in penalties'],relevant
8,hsbc fined $85 million for lax anti-money-laundering ...,https://www.wsj.com/articles/hsbc-fined-85-million-for-lax-anti-money-laundering-controls-11639734739,https://www.wsj.com › articles › hsbc-fined-85-million-...,hsbc hsbc -1.02%decrease; red down pointing triangle holdings plc was fined for inadequate anti-money-laundering controls that the ...,"['HSBC HSBC', 'Holdings PLC', 'money', 'laundering']",relevant
9,ex-10.1 - sec.gov,https://www.sec.gov/Archives/edgar/data/83246/000119312512499980/d453978dex101.htm,https://www.sec.gov › Archives › edgar › data,hsbc bank usa has revamped its kyc program and now treats hsbc group affiliates ... anti-money laundering standards available in any location where the hsbc ...,"['HSBC Group', 'money laundering']

In [8]:
training[(training['snippet'].isna()) == True]

,title,link,displayed_link,snippet,snippet_highlighted_words,label
15,hsbc | violation tracker - good jobs first,https://violationtracker.goodjobsfirst.org/parent/hsbc,https://violationtracker.goodjobsfirst.org › parent › hsbc,NaN,NaN,not relevant


In [9]:
training.drop([15], inplace=True)

In [10]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, 
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
def tokenizer_depunctuation(row):
  tokens = word_tokenize(row['snippet'])
  return [word.lower() for word in tokens if word.isalpha()]

training['snippet_tokenized'] = training.apply(tokenizer_depunctuation,axis=1)

In [12]:
# lemmatizing function

lemmatizer = WordNetLemmatizer() 

def lemmatizer_with_pos(row):
  return [lemmatizer.lemmatize(word,get_wordnet_pos(word)) for word in row['snippet_tokenized']]

training['snippet_lemmatized'] = training.apply(lemmatizer_with_pos,axis=1)

In [13]:
# removing stopwords

def remove_sw(row):
  return list(set(row['snippet_lemmatized']).difference(stopwords.words()))

training['snippet_no_stopwords'] = training.apply(remove_sw,axis=1)

In [14]:
# restructuring function

def re_structure(row):
  return " ".join(row['snippet_no_stopwords'])

training['snippet_clean'] = training.apply(re_structure,axis=1)

In [15]:
training.head()

,title,link,displayed_link,snippet,snippet_highlighted_words,label,snippet_tokenized,snippet_lemmatized,snippet_no_stopwords,snippet_clean
0,hsbc holdings plc. and hsbc bank usa n.a. admit to anti ...,https://www.justice.gov/opa/pr/hsbc-holdings-plc-and-hsbc-bank-usa-na-admit-anti-money-laundering-and-sanctions-violations,https://www.justice.gov › Office of Public Affairs › News,"according to court documents, hsbc bank usa violated the bsa by failing to maintain an effective anti-money laundering program and to conduct ...",['violated the BSA'],relevant,"[according, to, court, documents, hsbc, bank, usa, violated, the, bsa, by, failing, to, maintain, an, effective, laundering, program, and, to, conduct]","[accord, to, court, document, hsbc, bank, usa, violate, the, bsa, by, fail, to, maintain, an, effective, laundering, program, and, to, conduct]","[laundering, court, fail, bsa, bank, program, usa, violate, effective, conduct, document, hsbc, accord, maintain]",laundering court fail bsa bank program usa violate effective conduct document hsbc accord maintain
1,hsbc's money laundering scandal - investopedia,https://www.investopedia.com/stock-analysis/2013/investing-news-for-jan-29-hsbcs-money-laundering-scandal-hbc-scbff-ing-cs-rbs0129.aspx,https://www.investopedia.com › Investing › Stocks,hsbc holdings' (hsbc) agreement to pay a $1.9 billion fine to regulators for serving as a middleman for mexican drug cartels and enter into a deferred ...,['pay a $1.9 billion fine'],relevant,"[hsbc, holdings, hsbc, agreement, to, pay, a, billion, fine, to, regulators, for, serving, as, a, middleman, for, mexican, drug, cartels, and, enter, into, a, deferred]","[hsbc, holding, hsbc, agreement, to, pay, a, billion, fine, to, regulator, for, serve, a, a, middleman, for, mexican, drug, cartel, and, enter, into, a, defer]","[middleman, cartel, defer, billion, pay, regulator, agreement, serve, holding, hsbc, enter, mexican, fine]",middleman cartel defer billion pay regulator agreement serve holding hsbc enter mexican fine
2,fed ends decade-long enforcement action against hsbc,https://www.reuters.com/business/finance/us-fed-terminates-enforcement-action-against-hsbc-2022-09-02/,https://www.reuters.com › business › finance › us-fed-ter...,"london-headquartered hsbc was accused in 2012 of degenerating into a ""preferred financial institution"" for mexican and colombian drug cartels ...",['accused in 2012 of degenerating'],relevant,"[hsbc, was, accused, in, of, degenerating, into, a, preferred, financial, institution, for, mexican, and, colombian, drug, cartels]","[hsbc, be, accuse, in, of, degenerate, into, a, prefer, financial, institution, for, mexican, and, colombian, drug, cartel]","[cartel, prefer, degenerate, accuse, institution, colombian, financial, hsbc, mexican]",cartel prefer degenerate accuse institution colombian financial hsbc mexican
3,hsbc fined £64m for anti-money laundering failings - bbc,https://www.bbc.com/news/business-59689581,https://www.bbc.com › news › business-59689581,"banking giant hsbc has been fined £63.9m by the uk's financial regulator for ""unacceptable failings"" of its anti-money laundering systems. the ...",['has been fined £63.9m by'],relevant,"[banking, giant, hsbc, has, been, fined, by, the, uk, financial, regulator, for, unacceptable, failings, of, its, laundering, systems, the]","[banking, giant, hsbc, have, be, fin, by, the, uk, financial, regulator, for, unacceptable, failing, of, it, laundering, system, the]","[uk, laundering, regulator, system, giant, failing, financial, hsbc, banking, unacceptable, fin]",uk laundering regulator system giant failing financial hsbc banking unacceptable fin
4,hsbc moved vast sums of dirty money after paying record ...,https://www.icij.org/investigations/fincen-files/hsbc-moved-vast-sums-of-dirty-money-after-paying-record-laundering-fine/,https://www.icij.org › investigations › fincen-files › hsbc-...,hsbc moved vast sums of dirty money after paying record laundering fine ...

In [16]:
vect = TfidfVectorizer()
 
X = vect.fit_transform(training['snippet_clean'])

In [17]:
X

<301x1139 sparse matrix of type '<class 'numpy.float64'>'
	with 3523 stored elements in Compressed Sparse Row format>

In [18]:
y = training['label']

In [19]:
svm_model = SVC(kernel='linear', C=1.0)
svm_model.fit(X, y)

SVC(kernel='linear')

In [20]:
svm_model.score(X, y)

0.9867109634551495

In [21]:
# need 2 classes: more articles that don't relate to AML failures?

In [22]:
with open('vect.pkl', 'wb') as file:
    pickle.dump(vect, file)

In [23]:
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(svm_model, file)